<a href="https://colab.research.google.com/github/tsparaskevas/DataJournalism/blob/main/Scraping%20(news247)%20Solution/scraping_news247.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Μεθοδολογία κατασκευής scrapper για όλα τα άρθρα ενός site (με requests και BeautifulSoup) - [news247.gr]

##**Στόχοι**: 

###1. Να μαζέψω όλα τα άρθρα που δημοσιεύτηκαν σε μια μέρα στο news247

###2. Να φτιάξω κώδικα για τακτική συλλογή και αποθήκευση άρθρων από το news247



---



---



##**Στόχος 1**: Να μαζέψω όλα τα άρθρα που δημοσιεύτηκαν σε μια μέρα στο news247

**Πώς θα το κάνω**:
1. Θα δω με ποιο τρόπο εμφανίζεται η ροή των ειδήσεων στο news247.gr
2. Θα κατεβάσω την html των σελίδων που εμφανίζονται τα teaser articles
3. Θα δω πώς είναι δομημένο το περιεχόμενο των σελίδων με τα teaser articles ώστε να αναζητήσω με τα κατάλληλα tags τα δεδομένα που θέλω για το κάθε teaser article
4. Θα συλλέξω τα urls των teaser articles (και όλα τα δεδομένα τους: τίτλο, summary, section, date)
5. Θα δω πώς είναι δομημένο το περιεχόμενο της σελίδας του πλήρους άρθρου ώστε να αναζητήσω με τα κατάλληλα tags τα δεδομένα που θέλω για το κάθε άρθρο
6. θα κατεβάσω τις html όλων των σελίδων των ολόκληρων άρθρων
7. Θα συλλέξω όλα τα δεδομένα των άρθρων (Τίτλος, εικόνα [alt και src], section, lead, body και author)

####1. Πώς εμφανίζεται η ροή των ειδήσεων στο news247;

Το url της αρχικής σελίδας του news247 είναι https://www.news247.gr/

Η ροή των ειδήσεων υπάρχει στο menu item "Ειδήσεις": https://www.news247.gr/latest/

Στη σελίδα εμφανίζονται τα teasers των δημοσιοποιημένων άρθρων το ένα κάτω από το άλλο με φθήνουσα ημερολογιακή σειρά (τα πιο πρόσφατα εμφανίζονται πάνω-πάνω). Η σελίδα περιέχει 40 άρθρα και στο τέλος τους υπάρχουν κουμπιά για επόμενες σελίδες (με παλαιότερα άρθρα).

Πατώντας το κουμπί για τη δεύτερη σελίδα των Ειδήσεων, βλέπω ότι το url ειναι: https://www.news247.gr/latest/?pages=2. Για κάθε επόμενη σελίδα, αλλάζει στο url ο αριθμός στο ?pages=. Αν γράψω στο address bar του browser https://www.news247.gr/latest/?pages=1, βλέπω ότι εμφανίζεται η πρώτη σελίδα των Ειδήσεων. Αν βάλω στο pages=100, βλέπω ότι πάω σε ειδήσεις περίπου ένα μήνα πίσω και αν βάλω pages=200, πάω περίπου 2 μήνες πίσω. Αν βάλω pages=10000 πάω στο 2008. Παίζοντας με τον αριθμό στο pages=, βρίσκω ότι η τελευταία σελίδα στις Ειδήσεις του news247 είναι στο pages=10453 (Ιούνιος του 2006).

**Άρα**: 
* Το βασικό url για τη ροή των ειδήσεων στο news247 είναι: https://www.news247.gr/latest/?pages=. 
* Συμπληρώνοντας το βασικό url με αριθμούς από το 1 και πάνω μεταφέρομαι σε επόμενες σελίδες που περιέχουν παλαιότερα άρθρα.
* Οι ειδήσεις μιας μέρας καταλαμβάνουν περίπου 4 σελίδες.
* Για να είμαι σίγουρος ότι θα πάρω τις ειδήσεις του τελευταίου 24ωρου πρέπει να μπω στις σελίδες με page= από 1 έως 5.

####2. Κατέβασμα των 5 πρώτων σελίδων από τις "Ειδήσεις" του news247

Εισαγωγή βιβλιοθηκών

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Κατέβασμα των σελίδων και αποθήκευσή τους σε μια λίστα

In [ ]:
main_url = "https://www.news247.gr/latest/?pages=" # το βασικό url των σελίδων με τα teaser articles

teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

for i in range(1, 6): # θέλουμε 5 σελίδες, άρα ζητάμε από τη λούπα να χρησιμοποιεί αριθμούς από το 1 έως το 6 (το 6 δεν περιλαμβάνεται)
  page_url = main_url + str(i) # ορισμός του url για καθεμιά από τις σελίδες των Ειδήσεων (προσθέτω στο τέλος του pages=, σαν string, τον εκάστοτε αριθμό i που παράγει η for loop)
  page = requests.get(page_url) # κστέβασμα της σελίδας
  teaser_soup = BeautifulSoup(page.text, 'html.parser') # μετατροπή της κατεβασμένης σελίδας σε html
  print(f"Now Scraping page: {i}") # εκτύπωση της ερασίας που εκτελείται για να πληροφορούμαι ότι δουλεύει
  teaser_soups_list.append(teaser_soup) # αποθήκευση της κάθε σελίδας που κατεβαίνει στη λίστα με όλες τις σελίδες που φτιάξαμε στην αρχή
  time.sleep(2) # παύση 2" ανάμεσα στα requests από σελίδα σε σελίδα

Now Scraping page: 1
Now Scraping page: 2
Now Scraping page: 3
Now Scraping page: 4
Now Scraping page: 5


In [ ]:
#print(soup.prettify())

####3. Πώς είναι δομημένο το περιεχόμενο των σελίδων με τα teaser articles;
(ώστε να αναζητήσω με τα κατάλληλα tags τα δεδομένα που θέλω για το κάθε teaser article)

Ανοίγοντας τη σελίδα https://www.news247.gr/latest/?pages=1 στον browser, κάνω inspect στο πρώτο teaser article (δεξί κλικ στο πρώτο άρθρο -> Inspect).

Βλέπω ότι το περιεχόμενο ολόκληρου του box του άρθρου είναι μέσα σε ένα <article> tag, με class="teaser__article article article_left ".

Θα δοκιμάσω να δω τι μου φέρνει το find με την επιλογή: `('article', {'class': 'teaser__article article article_left '})`

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '})

<article class="teaser__article article article_left " data-position="1">
<figure>
<a class="article__image" href="https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html" title="Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία">
<img alt="Γιατροί σε εργαστήριο της Γαλλίας" class=" lazy " data-src="https://www.news247.gr/img/2629/9637672/583000/w140/140/untitled_1.jpg" height="84" style="height:auto;" title="Γιατροί σε εργαστήριο της Γαλλίας" width="140"/>
</a>
</figure>
<h2 class="article__title bold">
<a href="https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html" title="Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία">Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία</a>
</h2>
<div class="article__summary">
<a class="article__section article__tag_section" href="https://www.news247.gr/eylogia-ton-pithikon" title="Ευλογιά των Πιθήκων">ΕΥΛ

**Εξήγηση του κώδικα**: `teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '})`

Οι 5 html σελίδες των teaser articles που κατεβάσαμε είναι αποθηκευμένες στη λίστα soups_list.

Με το `teaser_soups_list[0]` ζητάμε την πρώτη html (δηλαδή της σελίδας με page=1).

Με το `find('article', {'class': 'teaser__article article article_left '})` ζητάμε την πρώτη εγγραφή με tag `<article>` και `class='teaser__article article article_left '` (δηλαδή το πρώτο teaser article).

Εναλλακτικά, θα μπορούσαμε να ζητήσουμε όλες τις εγγραφές (με `find_all`) και μετά την πρώτη (με `[0]`). Δηλαδή, το ίδιο αποτέλεσμα, θα μπορούσαμε να το πάρουμε με την εντολή: 

`teaser_soups_list[0].find_all('article', {'class': 'teaser__article article article_left '})[0]`

Όπως βλέπουμε από το αποτέλεσμα της εντολής που τρέξαμε παραπάνω, τα δεδομένα που θέλουμε, βρίσκονται στα παρακάτω tags:

* **Title**: `<h2> <a> 'title'`
* **Url**: `<h2> <a> 'href'`
* **Lead**: `<p> text`
* **Section**: `<a> class='article__section article__tag_section' 'title'`
* **Datetime**: `<time> 'datetime'`

Ας τα δοκιμάσουμε.

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '}).find('h2').find('a')['title'] # Title

'Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία'

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '}).find('h2').find('a')['href'] # Url

'https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html'

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '}).find('p').text # Lead

'Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο κρούσμα της νέας νόσου. Παγκόσμια ανησυχία.'

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '}).find('a', {'class': 'article__section article__tag_section'})['title'] # Section tag

'Ευλογιά των Πιθήκων'

In [ ]:
teaser_soups_list[0].find('article', {'class': 'teaser__article article article_left '}).find('time')['datetime'] # Datetime

'2022-05-20 12:12'

####4. Συλλογή των δεδομένων που θέλουμε από όλα τα teaser articles όλων των σελίδων που κατεβάσαμε από τις Ειδήσεις του news247.gr

In [ ]:
# Μάζεμα των δεδομένων κάθε teaser άρθρου από όλα τα teaser άρθρα όλων των σελίδων που φέραμε από τις Ειδήσεις 

teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
for teaser_soup in teaser_soups_list:
  teaser_articles = teaser_soup.find_all('article', {'class': 'teaser__article article article_left '})
  for teaser_article in teaser_articles:
    teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
    h2_a = teaser_article.find('h2').find('a')
    if h2_a:
      teaser_article_dict['Title'] = h2_a['title']
      teaser_article_dict['Url'] = h2_a['href']
    lead = teaser_article.find('p')
    if lead:
      teaser_article_dict['Lead'] = lead.text
    section = teaser_article.find('a', {'class': 'article__section article__tag_section'})
    if section:
      teaser_article_dict['Section'] = section['title']
    datetime = teaser_article.find('time')
    if datetime:
      teaser_article_dict['Datetime'] = datetime['datetime']
    teaser_articles_list.append(teaser_article_dict)

# Δημιουργία του dataframe
news247_latest_teaser_articles_df = pd.DataFrame(teaser_articles_list)
news247_latest_teaser_articles_df

,Title,Url,Lead,Section,Datetime
0,Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσ...,https://www.news247.gr/kosmos/eylogia-ton-pith...,Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο...,Ευλογιά των Πιθήκων,2022-05-20 12:12
1,Ο Ούλριχ Ράσε στην Επίδαυρο: Γιατί οι παραστάσ...,https://www.news247.gr/politismos/theatro/o-oy...,Η σκηνή που θα στηθεί στις 22 και 23 Ιουλίου π...,Φεστιβάλ Αθηνών και Επιδαύρου,2022-05-20 11:50
2,Δίκη Τοπαλούδη: Κανένα ελαφρυντικό για τους δο...,https://www.news247.gr/koinonia/diki-topaloydi...,"Με ομόφωνη απόφαση, το Μικτό Ορκωτό Δικαστήριο...",Ελένη Τοπαλούδη,2022-05-20 11:42
3,Θυμίζεις κάτι από Πρέβελη: Το άγνωστο φοινικόδ...,https://www.news247.gr/taxidi/thymizeis-kati-a...,"Περιηγηθείτε σε ένα καταπράσινο φοινικόδασος, ...",drone,2022-05-20 11:40
4,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/koinonia/rethymno-epith...,Αγνωστοι εξαπέλυσαν επίθεση με αυγό στο γραμμα...,Πανεπιστήμια,2022-05-20 11:39
...,...,...,...,...,...
195,Survivor Trailer: Χάος στην παραλία - Ποιος απ...,https://www.news247.gr/psixagogia/tileorasi/su...,Τεράστια θα είναι η ένταση στο αποψινό επεισόδ...,Survivor Αποχώρηση,2022-05-19 08:52
196,Τροχαίο ατύχημα με τρία αυτοκίνητα και τρεις τ...,https://www.news247.gr/koinonia/trochaio-atych...,Το περιστατικό συνέβη λίγο πριν τα μεσάνυχτα σ...,Τροχαία Ατυχήματα,2022-05-19 08:40
197,Σκύδρα: Στο Ιπποκράτειο 7χρονος - Έπεσε επάνω ...,https://www.news247.gr/koinonia/skydra-sto-ipp...,"Σοβαρό ατύχημα σημειώθηκε στη Σκύδρα Πέλλας, ό...",Θεσσαλονίκη,2022-05-19 08:34
198,"Τζορτζ Μπους: ""Αδικαιολόγητη και βάναυση η εισ...",https://www.news247.gr/kosmos/gkafa-olkis-o-tz...,Ο Τζορτζ Μπους έγινε αντικείμενο έντονου σχολι...,Πόλεμος στην Ουκρανία,2022-05-19 08:22


####5. Πώς είναι δομημένο το περιεχόμενο της σελίδας του πλήρους άρθρου;
(ώστε να αναζητήσω με τα κατάλληλα tags τα δεδομένα που θέλω για το κάθε άρθρο)


Τα Urls των σελίδων ολόκληρων των άρθρων βρίσκονται πλέον στο dataframe με τα teaser articles που έχουμε φτιάξει.

Επομένως αν θέλουμε να κατεβάσουμε τη σελίδα του πρώτου άρθρου, θα πάρουμε το url της από το πρώτο row του dataframe από τη στήλη 'Url'.

In [ ]:
test_article_url = news247_latest_teaser_articles_df.iloc[[0]]['Url'].item() # με το item() παίρνω το string του Url

test_response = requests.get(test_article_url)
test_soup = BeautifulSoup(test_response.text, 'html.parser')

Θέλω να δω ποιο είναι το url του πρώτου άρθρου που έχω κατεβάσει για να το ανοίξω στον browser και να το κάνω inspect.

In [ ]:
test_article_url

'https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html'

Ανοίγοντας τη σελίδα https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html στον browser, κάνω inspect (δεξί κλικ στον τίτλο του άρθρου -> Inspect).

Βλέπω ότι το περιεχόμενο ολόκληρου του box του άρθρου είναι μέσα σε ένα `<article> tag`, με `id="article-ifo"`. Όμως μέσα στο `<article>` είναι και πάρα πολλά σκουπίδια που δεν τα θέλω.

Το άρθρο καθεαυτό φαίνεται να είναι μέσα σε ένα `<div>` με `class="article-body "`.

Όμως το Section είναι έξω από αυτό το `<div>`, σ'ένα άλλο `<div>` με `class="article-head"`.


Θα δοκιμάσω να δω τι μου φέρνει το find με την επιλογή: `('div', {'class': 'article-body'})`

In [ ]:
test_soup.find('div', {'class': 'article-body'})

<div class="article-body ">
<div class="article-body__article-head-box">
<h1 class="article-body__article-headline">
					Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία</h1>
</div>
<figure class="article-body__picture">
<img alt="Γιατροί σε εργαστήριο της Γαλλίας" height="381" src="https://www.news247.gr/img/4769/9637672/583000/w660/660/untitled_1.jpg" style="height:auto;" title="Γιατροί σε εργαστήριο της Γαλλίας" width="660"/>
<figcaption class="article-body__picture-descr">
                            Γιατροί σε εργαστήριο της Γαλλίας<span class="article-body__credit">
<i aria-hidden="true" class="fa fa-camera"></i>
                                            AP PHOTO/JEAN-FRANCOIS BADIAS</span>
</figcaption>
</figure>
<div class="sociall_container">
<!-- Go to www.addthis.com/dashboard to customize your tools -->
<div class="addthis_inline_share_toolbox_cg60_ry40"></div>
</div>
<div class="ad-widget ad-widget_custom ">
<!--script src="//code.jquery.com/jquery-1.12.0

Αυτή η επιλογή περιέχει το άρθρο αλλά και πολλά σκουπίδια. Πάντως μέσα σ' αυτό το div υπάρχει στο `<h1>` ο Τίτλος και στο `<h2>` το Lead.

In [ ]:
# Title
test_soup.find('div', {'class': 'article-body '}).find('h1').text 

'\r\n\t\t\t\t\tΕυλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία'

Βλέπω ότι ο Τίτλος έχει splitlines και tabs (\r, \n, \t), τα οποία μπορώ να τα αφαιρώ.

In [ ]:
"".join(test_soup.find('div', {'class': 'article-body '}).find('h1').text.splitlines()).replace('\t', '')

'Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσμα και στη Γαλλία'

In [ ]:
# Lead
test_soup.find('div', {'class': 'article-body '}).find('h2').text 

'Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο κρούσμα της νέας νόσου. Παγκόσμια ανησυχία.'

Ψάχνοντας και δοκιμάζοντας με τον ίδιο τρόπο βρίσκω σε ποια tags είναι και τα υπόλοιπα στοιχεία του άρθρου που θέλω.

In [ ]:
# Section tag
test_soup.find('div', {'class': 'article-head'}).find('a')['title']

'Κόσμος'

In [ ]:
# Picture alt text
test_soup.find('figure', {'class': 'article-body__picture'}).find('img')['alt'] 

'Γιατροί σε εργαστήριο της Γαλλίας'

In [ ]:
test_soup.find('figure', {'class': 'article-body__picture'}).find('img')['src'] # Picture url

'https://www.news247.gr/img/4769/9637672/583000/w660/660/untitled_1.jpg'

In [ ]:
# Editor
test_soup.find('div', {'class': 'article-body__byline noeditor'}).find('a').text 

'news247'

In [ ]:
# Date
test_soup.find('span', {'class': 'article-body__date'}).find('time')['datetime'] 

'2022-05-20 12:12'

Πρόβλημα με το body (σώμα) του άρθρου.

In [ ]:
# Paragraphs
test_soup.find('div', {'class': 'article-body__body'}).find_all('p')

[<p>Ένα πρώτο επιβεβαιωμένο κρούσμα της<a href="https://www.news247.gr/eylogia-ton-pithikon" rel="Follow" target="_self"> ευλογιάς των πιθήκων</a> εντοπίστηκε χθες, Πέμπτη, στην περιφέρεια Ιλ-ντε-Φρανς, ανακοίνωσε σήμερα η Γενική Διεύθυνση Υγείας της <a href="https://www.news247.gr/gallia" rel="Follow" target="_self">Γαλλίας</a>.</p>,
 <p>Πρόκειται για έναν άνδρα ηλικίας 29 ετών χωρίς προηγούμενο ταξιδιού σε χώρα όπου κυκλοφορεί ο ιός, διευκρίνισαν οι υγειονομικές αρχές.</p>,
 <p>Αφότου θεωρήθηκε ότι μπορεί να είχε μολυνθεί, έλαβε ιατρική φροντίδα και, καθώς η κατάστασή του δεν είναι σοβαρή, έχει τεθεί σε απομόνωση στο σπίτι του.</p>,
 <p>Δεύτερο κρούσμα ευλογιάς των πιθήκων καταγράφηκε στο Βέλγιο, σύμφωνα με τον ιολόγο Μαρκ Βαν Ρανστ.</p>,
 <p>"Δεύτερο κρούσμα διαγνώστηκε χθες (Πέμπτη) το βράδυ στο εργαστήριό μας στο Λουβέν σε δείγμα άνδρα από την περιφέρεια της Φλαμανδικής Μπραμπάντ", ανέφερε ο Μαρκ Βαν Ρανστ. Το πρώτο κρούσμα είχε εντοπιστεί στην Αμβέρσα.</p>,
 <p><i><b>Ακολουθήστε 

**Παρατήρηση**:

Όπως φαίνεται στο άρθρο στον browser (https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html), ανάμεσα στις παραγράφους, υπάρχει κι ένας υπότιτλος (`<h2>`). Αν τον πάρω μόνο του δεν θα τον έχω μέσα στη ροή του κειμένου.

Mπορώ όμως με το find_all, να ζητήσω ταυτόχρονα τα `<p>` και τα `<h2>`.

In [ ]:
test_soup.find('div', {'class': 'article-body__body'}).find_all(['p', 'h2'])

[<p>Ένα πρώτο επιβεβαιωμένο κρούσμα της<a href="https://www.news247.gr/eylogia-ton-pithikon" rel="Follow" target="_self"> ευλογιάς των πιθήκων</a> εντοπίστηκε χθες, Πέμπτη, στην περιφέρεια Ιλ-ντε-Φρανς, ανακοίνωσε σήμερα η Γενική Διεύθυνση Υγείας της <a href="https://www.news247.gr/gallia" rel="Follow" target="_self">Γαλλίας</a>.</p>,
 <p>Πρόκειται για έναν άνδρα ηλικίας 29 ετών χωρίς προηγούμενο ταξιδιού σε χώρα όπου κυκλοφορεί ο ιός, διευκρίνισαν οι υγειονομικές αρχές.</p>,
 <p>Αφότου θεωρήθηκε ότι μπορεί να είχε μολυνθεί, έλαβε ιατρική φροντίδα και, καθώς η κατάστασή του δεν είναι σοβαρή, έχει τεθεί σε απομόνωση στο σπίτι του.</p>,
 <h2>Δεύτερο κρούσμα στο Βέλγιο</h2>,
 <p>Δεύτερο κρούσμα ευλογιάς των πιθήκων καταγράφηκε στο Βέλγιο, σύμφωνα με τον ιολόγο Μαρκ Βαν Ρανστ.</p>,
 <p>"Δεύτερο κρούσμα διαγνώστηκε χθες (Πέμπτη) το βράδυ στο εργαστήριό μας στο Λουβέν σε δείγμα άνδρα από την περιφέρεια της Φλαμανδικής Μπραμπάντ", ανέφερε ο Μαρκ Βαν Ρανστ. Το πρώτο κρούσμα είχε εντοπιστεί στη

Με το find_all παίρνω μια λίστα με όλες τις παραγράφους και τους υπότιτλους. Αφού εγώ θέλω όλο το body σαν ένα string, μπορώ να συνενώσω όλα τα στοιχεία.

Πρώτα όμως πρέπει να πάρω μόνο το text.

Επίσης, παρατηρώ ότι το τελευταίο p είναι άσχετο με το άρθρο και επιβεβαιώνω επισκοπώντας και άλλες σελίδες άρθρων στον browser ότι αυτό ισχύει για όλα τα άρθρα. Άρα μπορώ να πετάω το τελευταίο `<p>`.

In [ ]:
# Μάζεμα του text όλων των p και h2 σε μία λίστα
test_body = []
for p in test_soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'):
  test_p_text = p.text
  test_body.append(test_p_text)
test_body[:-1]

['Ένα πρώτο επιβεβαιωμένο κρούσμα της ευλογιάς των πιθήκων εντοπίστηκε χθες, Πέμπτη, στην περιφέρεια Ιλ-ντε-Φρανς, ανακοίνωσε σήμερα η Γενική Διεύθυνση Υγείας της Γαλλίας.',
 'Πρόκειται για έναν άνδρα ηλικίας 29 ετών χωρίς προηγούμενο ταξιδιού σε χώρα όπου κυκλοφορεί ο ιός, διευκρίνισαν οι υγειονομικές αρχές.',
 'Αφότου θεωρήθηκε ότι μπορεί να είχε μολυνθεί, έλαβε ιατρική φροντίδα και, καθώς η κατάστασή του δεν είναι σοβαρή, έχει τεθεί σε απομόνωση στο σπίτι του.',
 'Δεύτερο κρούσμα ευλογιάς των πιθήκων καταγράφηκε στο Βέλγιο, σύμφωνα με τον ιολόγο Μαρκ Βαν Ρανστ.',
 '"Δεύτερο κρούσμα διαγνώστηκε χθες (Πέμπτη) το βράδυ στο εργαστήριό μας στο Λουβέν σε δείγμα άνδρα από την περιφέρεια της Φλαμανδικής Μπραμπάντ", ανέφερε ο Μαρκ Βαν Ρανστ. Το πρώτο κρούσμα είχε εντοπιστεί στην Αμβέρσα.']

In [ ]:
# Συνένωση των μεμονωμένων στοιχείων του κειμένου σε ένα string και μετατροπή του τυχόν διπλού κενού που μπορεί να δημιουργείται σε κάποιες περιπτώσεις
test_body_text = ' '.join(test_body[:-1]).replace("  ", " ")
test_body_text

'Ένα πρώτο επιβεβαιωμένο κρούσμα της ευλογιάς των πιθήκων εντοπίστηκε χθες, Πέμπτη, στην περιφέρεια Ιλ-ντε-Φρανς, ανακοίνωσε σήμερα η Γενική Διεύθυνση Υγείας της Γαλλίας. Πρόκειται για έναν άνδρα ηλικίας 29 ετών χωρίς προηγούμενο ταξιδιού σε χώρα όπου κυκλοφορεί ο ιός, διευκρίνισαν οι υγειονομικές αρχές. Αφότου θεωρήθηκε ότι μπορεί να είχε μολυνθεί, έλαβε ιατρική φροντίδα και, καθώς η κατάστασή του δεν είναι σοβαρή, έχει τεθεί σε απομόνωση στο σπίτι του. Δεύτερο κρούσμα ευλογιάς των πιθήκων καταγράφηκε στο Βέλγιο, σύμφωνα με τον ιολόγο Μαρκ Βαν Ρανστ. "Δεύτερο κρούσμα διαγνώστηκε χθες (Πέμπτη) το βράδυ στο εργαστήριό μας στο Λουβέν σε δείγμα άνδρα από την περιφέρεια της Φλαμανδικής Μπραμπάντ", ανέφερε ο Μαρκ Βαν Ρανστ. Το πρώτο κρούσμα είχε εντοπιστεί στην Αμβέρσα.'

####6. Κατέβασμα των html όλων των σελίδων των ολόκληρων άρθρων


Όπως είπαμε παραπάνω, τα urls των άρθρων που θέλουμε να κατεβάσουμε βρίσκονται στη στήλη 'Url' του dataframe news247_latest_teaser_articles_df

In [ ]:
# Κάνω πρώτα μια δοκιμή με τα πρώτα 5  urls
full_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των ολόκληρων άρθρων
nbr = 0 # βάζω αρίθμηση για να παρακολουθώ το scraping

for article_url in news247_latest_teaser_articles_df['Url'][:5]: 
  nbr += 1 
  full_response = requests.get(article_url) 
  full_soup = BeautifulSoup(full_response.text, 'html.parser')
  full_soups_list.append(full_soup)
  time.sleep(2)
  print(f'Scraping article {nbr}')

Scraping article 1
Scraping article 2
Scraping article 3
Scraping article 4
Scraping article 5


Εφόσον δουλεύει, μπορώ να τα κατεβάσω όλα

In [ ]:
full_soups_list = [] 
nbr = 0 

for article_url in news247_latest_teaser_articles_df['Url']: 
  nbr += 1 
  full_response = requests.get(article_url) 
  full_soup = BeautifulSoup(full_response.text, 'html.parser')
  full_soups_list.append(full_soup)
  time.sleep(2)
  print(f'Scraping article {nbr}')

Scraping article 1
Scraping article 2
Scraping article 3
Scraping article 4
Scraping article 5
Scraping article 6
Scraping article 7
Scraping article 8
Scraping article 9
Scraping article 10
Scraping article 11
Scraping article 12
Scraping article 13
Scraping article 14
Scraping article 15
Scraping article 16
Scraping article 17
Scraping article 18
Scraping article 19
Scraping article 20
Scraping article 21
Scraping article 22
Scraping article 23
Scraping article 24
Scraping article 25
Scraping article 26
Scraping article 27
Scraping article 28
Scraping article 29
Scraping article 30
Scraping article 31
Scraping article 32
Scraping article 33
Scraping article 34
Scraping article 35
Scraping article 36
Scraping article 37
Scraping article 38
Scraping article 39
Scraping article 40
Scraping article 41
Scraping article 42
Scraping article 43
Scraping article 44
Scraping article 45
Scraping article 46
Scraping article 47
Scraping article 48
Scraping article 49
Scraping article 50
Scraping 

####Συλλογή όλων των δεδομένων των άρθρων (Τίτλος, εικόνα [alt και src], section, lead, body και author)

**Title**

test_soup.find('div', {'class': 'article-body '}).find('h1').text 

"".join(test_soup.find('div', {'class': 'article-body '}).find('h1').text.splitlines()).replace('\t', '')

**Lead**

test_soup.find('div', {'class': 'article-body '}).find('h2').text 

**Section tag**

test_soup.find('div', {'class': 'article-head'}).find('a')['title']

**Picture alt text**

test_soup.find('figure', {'class': 'article-body__picture'}).find('img')['alt']

**Picture url**

test_soup.find('figure', {'class': 'article-body__picture'}).find('img')['src']

**Editor**

test_soup.find('div', {'class': 'article-body__byline noeditor'}).find('a').text 

**Date**
test_soup.find('span', {'class': 'article-body__date'}).find('time')['datetime'] 



In [ ]:
# Μάζεμα των δεδομένων του άρθρου
full_articles_list = []
nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του news247_latest_teaser_articles_df, να φέρνω και το αντίστοιχο url του άρθρου
for soup in full_soups_list:
  print(f"working on article {news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item()}") # τυπώνω το url του άρθρου για να το επισκοπήσω αν τυχόν βαρέσει ο κώδικας
  full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
  title = soup.find('div', {'class': 'article-body '}).find('h1') # Title
  if title:
    full_article_dict['Title'] = "".join(title.text.splitlines()).replace('\t', '')
  full_article_dict['Article url'] = news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου
  section = soup.find('div', {'class': 'article-head'}).find('a') # Section
  if section:
    full_article_dict['Section'] = section['title']
  image = soup.find('figure', {'class': 'article-body__picture'}) # Image
  if image:
    full_article_dict['Image alt text'] = image.find('img')['alt'] 
    full_article_dict['Image url'] = image.find('img')['src']
  lead = soup.find('div', {'class': 'article-body '}).find('h2') # Lead
  if lead:
    full_article_dict['Lead'] = "".join(lead.text.splitlines())
  editor = soup.find('div', {'class': 'article-body__byline noeditor'}).find('a') # Editor
  if editor:
    full_article_dict['Editor'] = editor.text
  time = soup.find('span', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  body = []
  for p in soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'): # Body
    p_text = p.text
    if p_text:
      body.append(p_text)
      body_text = ' '.join(body[:-1]).replace("  ", " ")
      body_text = "".join(body_text.splitlines())
      full_article_dict['Body'] = body_text
  full_articles_list.append(full_article_dict)
  nbr += 1

working on article https://www.news247.gr/kosmos/eylogia-ton-pithikon-proto-epivevaiomeno-kroysma-kai-sti-gallia.9637679.html
working on article https://www.news247.gr/politismos/theatro/o-oylrich-rase-stin-epidayro-giati-oi-parastaseis-toy-thymizoyn-laiko-proskynima-se-techno-club.9637610.html


AttributeError: ignored

Ο κώδικας χτυπάει στον Editor, στο 2ο άρθρο. Ανοίγω το url στον browser και βλέπω ότι εδώ υπάρχει το όνομα του αρθρογράφου (αντί για το σύνηθες news247), το οποίο έχει άλλο class. 

Επιπλέον, σκέφτομαι ότι το news247 σαν όνομα Editor, δεν το χρειάζομαι.

Επομένως, διορθώνω τον κώδικα καταγράφοντας μόνο τον Author αν υπάρχει και ξαναδοκιμάζω.

In [ ]:
# Μάζεμα των δεδομένων του άρθρου
full_articles_list = []
nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του news247_latest_teaser_articles_df, να φέρνω και το αντίστοιχο url του άρθρου
for soup in full_soups_list:
  print(f"working on article {news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item()}")
  full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
  title = soup.find('div', {'class': 'article-body '}).find('h1') # Title
  if title:
    full_article_dict['Title'] = "".join(title.text.splitlines()).replace('\t', '')
  full_article_dict['Article url'] = news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου
  section = soup.find('div', {'class': 'article-head'}).find('a') # Section
  if section:
    full_article_dict['Section'] = section['title']
  image = soup.find('figure', {'class': 'article-body__picture'}) # Image
  if image:
    full_article_dict['Image alt text'] = image.find('img')['alt'] 
    full_article_dict['Image url'] = image.find('img')['src']
  lead = soup.find('div', {'class': 'article-body '}).find('h2') # Lead
  if lead:
    full_article_dict['Lead'] = "".join(lead.text.splitlines())
  author = soup.find('div', {'class': 'article-body__author-name'}) # Author
  if author:
    full_article_dict['Author'] = "".join(author.text.splitlines())
  time = soup.find('span', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  body = []
  for p in soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'): # Body
    p_text = p.text
    if p_text:
      body.append(p_text)
      body_text = ' '.join(body[:-1]).replace("  ", " ")
      body_text = "".join(body_text.splitlines())
      full_article_dict['Body'] = body_text
  full_articles_list.append(full_article_dict)
  nbr += 1

Όλα φαίνεται ότι πήγαν καλά, οπότε φτιάχνω το dataframe από το full_articles_list και το επισκοπώ

In [ ]:
# Δημιουργία του dataframe
news247_latest_full_articles_df = pd.DataFrame(full_articles_list)
news247_latest_full_articles_df

,Title,Article url,Section,Image alt text,Image url,Lead,Datetime,Body,Author
0,Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσ...,https://www.news247.gr/kosmos/eylogia-ton-pith...,Κόσμος,Γιατροί σε εργαστήριο της Γαλλίας,https://www.news247.gr/img/4769/9637672/583000...,Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο...,2022-05-20 12:12,Ένα πρώτο επιβεβαιωμένο κρούσμα της ευλογιάς τ...,NaN
1,Ο Ούλριχ Ράσε στην Επίδαυρο: Γιατί οι παραστάσ...,https://www.news247.gr/politismos/theatro/o-oy...,Θέατρο,"Ο Γερμανός σκηνοθέτης, Ούλριχ Ράσε",https://www.news247.gr/img/752/9637612/600000/...,Η σκηνή που θα στηθεί στις 22 και 23 Ιουλίου π...,NaN,Πολυαναμενόμενη η παράσταση του καταξιωμένου Γ...,Γεωργία Οικονόμου
2,Δίκη Τοπαλούδη: Κανένα ελαφρυντικό για τους δο...,https://www.news247.gr/koinonia/diki-topaloydi...,Κοινωνία,Οι δύο κατηγορούμενοι για τη δολοφονία και το ...,https://www.news247.gr/img/36/9637565/787000/w...,"Με ομόφωνη απόφαση, το Μικτό Ορκωτό Δικαστήριο...",NaN,Την απόρριψη όλων των ελαφρυντικών περιστάσεων...,NaN
3,Θυμίζεις κάτι από Πρέβελη: Το άγνωστο φοινικόδ...,https://www.news247.gr/taxidi/thymizeis-kati-a...,Ταξίδι,Το άγνωστο φοινικόδασος της Αττικής,https://www.news247.gr/img/6760/9637609/291000...,"Περιηγηθείτε σε ένα καταπράσινο φοινικόδασος, ...",2022-05-20 11:40,Αν έχετε επισκεφτεί το φοινικόδασος στην μαγευ...,NaN
4,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/koinonia/rethymno-epith...,Κοινωνία,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/img/9061/9637620/890000...,Αγνωστοι εξαπέλυσαν επίθεση με αυγό στο γραμμα...,2022-05-20 11:39,Η ένταση που επικρατεί στο χώρο του Πανεπιστημ...,NaN
...,...,...,...,...,...,...,...,...,...
195,Survivor Trailer: Χάος στην παραλία - Ποιος απ...,https://www.news247.gr/psixagogia/tileorasi/su...,Τηλεόραση,Στιγμιότυπο από το Συμβούλιο του Νησιού του Su...,https://www.news247.gr/img/6139/9635912/679000...,Τεράστια θα είναι η ένταση στο αποψινό επεισόδ...,2022-05-19 08:52,Γεμάτο εντάσεις αναμένεται να είναι το νέο επε...,NaN
196,Τροχαίο ατύχημα με τρία αυτοκίνητα και τρεις τ...,https://www.news247.gr/koinonia/trochaio-atych...,Κοινωνία,Ασθενοφόρο του ΕΚΑΒ,https://www.news247.gr/img/9585/9629015/814000...,Το περιστατικό συνέβη λίγο πριν τα μεσάνυχτα σ...,NaN,Τροχαίο ατύχημα σημειώθηκε λίγο πριν τα μεσάνυ...,NaN
197,Σκύδρα: Στο Ιπποκράτειο 7χρονος - Έπεσε επάνω ...,https://www.news247.gr/koinonia/skydra-sto-ipp...,Κοινωνία,Το Ιπποκράτειο Νοσοκομείο Θεσσαλονίκης,https://www.news247.gr/img/6906/9558439/146000...,"Σοβαρό ατύχημα σημειώθηκε στη Σκύδρα Πέλλας, ό...",2022-05-19 08:34,Στο Ιπποκράτειο Νοσοκομείο της Θεσσαλονίκης νο...,NaN
198,"Τζορτζ Μπους: ""Αδικαιολόγητη και βάναυση η εισ...",https://www.news247.gr/kosmos/gkafa-olkis-o-tz...,Κόσμος,Ο Τζορτζ Ουόκερ Μπους,https://www.news247.gr/img/9619/9635890/302000...,Ο Τζορτζ Μπους έγινε αντικείμενο έντονου σχολι...,2022-05-19 08:22,"Κατά τη διάρκειας ομιλίας του στο Ντάλας, ο πρ...",NaN


####Επισκόπηση για τυχόν αστοχίες και Fine tuning

Βλέπω ότι κάποια άρθρα δεν έχουν Datetime. Ανοίγω το url του πρώτου στον browser.

In [ ]:
print(news247_latest_teaser_articles_df.iloc[[1]]['Url'].item())

https://www.news247.gr/politismos/theatro/o-oylrich-rase-stin-epidayro-giati-oi-parastaseis-toy-thymizoyn-laiko-proskynima-se-techno-club.9637610.html


Mε Inspect στην ημερομηνία βλέπω ότι όταν υπάρχει όνομα Author, η ημερομηνία δεν είναι σε span αλλά σε div. Συμπληρώνω τον κώδικα και ξαναδοκιμάζω

In [ ]:
# Μάζεμα των δεδομένων του άρθρου
full_articles_list = []
nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του news247_latest_teaser_articles_df, να φέρνω και το αντίστοιχο url του άρθρου
for soup in full_soups_list:
  print(f"working on article {news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item()}")
  full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
  title = soup.find('div', {'class': 'article-body '}).find('h1') # Title
  if title:
    full_article_dict['Title'] = "".join(title.text.splitlines()).replace('\t', '')
  full_article_dict['Article url'] = news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου
  section = soup.find('div', {'class': 'article-head'}).find('a') # Section
  if section:
    full_article_dict['Section'] = section['title']
  image = soup.find('figure', {'class': 'article-body__picture'}) # Image
  if image:
    full_article_dict['Image alt text'] = image.find('img')['alt'] 
    full_article_dict['Image url'] = image.find('img')['src']
  lead = soup.find('div', {'class': 'article-body '}).find('h2') # Lead
  if lead:
    full_article_dict['Lead'] = "".join(lead.text.splitlines())
  author = soup.find('div', {'class': 'article-body__author-name'}) # Author
  if author:
    full_article_dict['Author'] = "".join(author.text.splitlines())
  time = soup.find('span', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  time = soup.find('div', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  body = []
  for p in soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'): # Body
    p_text = p.text
    if p_text:
      body.append(p_text)
      body_text = ' '.join(body[:-1]).replace("  ", " ")
      body_text = "".join(body_text.splitlines())
      full_article_dict['Body'] = body_text
  full_articles_list.append(full_article_dict)
  nbr += 1

In [ ]:
# Δημιουργία του dataframe
news247_latest_full_articles_df = pd.DataFrame(full_articles_list)
news247_latest_full_articles_df

,Title,Article url,Section,Image alt text,Image url,Lead,Datetime,Body,Author
0,Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσ...,https://www.news247.gr/kosmos/eylogia-ton-pith...,Κόσμος,Γιατροί σε εργαστήριο της Γαλλίας,https://www.news247.gr/img/4769/9637672/583000...,Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο...,2022-05-20 12:12,Ένα πρώτο επιβεβαιωμένο κρούσμα της ευλογιάς τ...,NaN
1,Ο Ούλριχ Ράσε στην Επίδαυρο: Γιατί οι παραστάσ...,https://www.news247.gr/politismos/theatro/o-oy...,Θέατρο,"Ο Γερμανός σκηνοθέτης, Ούλριχ Ράσε",https://www.news247.gr/img/752/9637612/600000/...,Η σκηνή που θα στηθεί στις 22 και 23 Ιουλίου π...,2022-05-20 11:50,Πολυαναμενόμενη η παράσταση του καταξιωμένου Γ...,Γεωργία Οικονόμου
2,Δίκη Τοπαλούδη: Κανένα ελαφρυντικό για τους δο...,https://www.news247.gr/koinonia/diki-topaloydi...,Κοινωνία,Οι δύο κατηγορούμενοι για τη δολοφονία και το ...,https://www.news247.gr/img/36/9637565/787000/w...,"Με ομόφωνη απόφαση, το Μικτό Ορκωτό Δικαστήριο...",2022-05-20 11:42,Την απόρριψη όλων των ελαφρυντικών περιστάσεων...,NaN
3,Θυμίζεις κάτι από Πρέβελη: Το άγνωστο φοινικόδ...,https://www.news247.gr/taxidi/thymizeis-kati-a...,Ταξίδι,Το άγνωστο φοινικόδασος της Αττικής,https://www.news247.gr/img/6760/9637609/291000...,"Περιηγηθείτε σε ένα καταπράσινο φοινικόδασος, ...",2022-05-20 11:40,Αν έχετε επισκεφτεί το φοινικόδασος στην μαγευ...,NaN
4,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/koinonia/rethymno-epith...,Κοινωνία,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/img/9061/9637620/890000...,Αγνωστοι εξαπέλυσαν επίθεση με αυγό στο γραμμα...,2022-05-20 11:39,Η ένταση που επικρατεί στο χώρο του Πανεπιστημ...,NaN
...,...,...,...,...,...,...,...,...,...
195,Survivor Trailer: Χάος στην παραλία - Ποιος απ...,https://www.news247.gr/psixagogia/tileorasi/su...,Τηλεόραση,Στιγμιότυπο από το Συμβούλιο του Νησιού του Su...,https://www.news247.gr/img/6139/9635912/679000...,Τεράστια θα είναι η ένταση στο αποψινό επεισόδ...,2022-05-19 08:52,Γεμάτο εντάσεις αναμένεται να είναι το νέο επε...,NaN
196,Τροχαίο ατύχημα με τρία αυτοκίνητα και τρεις τ...,https://www.news247.gr/koinonia/trochaio-atych...,Κοινωνία,Ασθενοφόρο του ΕΚΑΒ,https://www.news247.gr/img/9585/9629015/814000...,Το περιστατικό συνέβη λίγο πριν τα μεσάνυχτα σ...,2022-05-19 08:40,Τροχαίο ατύχημα σημειώθηκε λίγο πριν τα μεσάνυ...,NaN
197,Σκύδρα: Στο Ιπποκράτειο 7χρονος - Έπεσε επάνω ...,https://www.news247.gr/koinonia/skydra-sto-ipp...,Κοινωνία,Το Ιπποκράτειο Νοσοκομείο Θεσσαλονίκης,https://www.news247.gr/img/6906/9558439/146000...,"Σοβαρό ατύχημα σημειώθηκε στη Σκύδρα Πέλλας, ό...",2022-05-19 08:34,Στο Ιπποκράτειο Νοσοκομείο της Θεσσαλονίκης νο...,NaN
198,"Τζορτζ Μπους: ""Αδικαιολόγητη και βάναυση η εισ...",https://www.news247.gr/kosmos/gkafa-olkis-o-tz...,Κόσμος,Ο Τζορτζ Ουόκερ Μπους,https://www.news247.gr/img/9619/9635890/302000...,Ο Τζορτζ Μπους έγινε αντικείμενο έντονου σχολι...,2022-05-19 08:22,"Κατά τη διάρκειας ομιλίας του στο Ντάλας, ο πρ...",NaN


Τώρα υπάρχει και το Datetime εκεί που έλειπε. Για ασφάλεια, θα τσεκάρω και άλλο ένα άρθρο που δεν είχε Datetime, αλλά δεν είχε και Author.

In [ ]:
print(news247_latest_teaser_articles_df.iloc[[2]]['Url'].item())

https://www.news247.gr/koinonia/diki-topaloydi-kanena-elafryntiko-gia-toys-dolofonoys-kai-viastes-tis-elenis.9637568.html


Βλέπω ότι έχει Author, που όμως είναι σε `<a>` tag που είναι μέσα σε ένα `<div>` με `class="article-body__byline"`. Διορθώνω τον κώδικα για να πιάνει και αυτό τον Author και ξαναδοκιμάζω.

In [ ]:
# Μάζεμα των δεδομένων του άρθρου
full_articles_list = []
nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του news247_latest_teaser_articles_df, να φέρνω και το αντίστοιχο url του άρθρου
for soup in full_soups_list:
  print(f"working on article {news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item()}")
  full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
  title = soup.find('div', {'class': 'article-body '}).find('h1') # Title
  if title:
    full_article_dict['Title'] = "".join(title.text.splitlines()).replace('\t', '')
  full_article_dict['Article url'] = news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου
  section = soup.find('div', {'class': 'article-head'}).find('a') # Section
  if section:
    full_article_dict['Section'] = section['title']
  image = soup.find('figure', {'class': 'article-body__picture'}) # Image
  if image:
    full_article_dict['Image alt text'] = image.find('img')['alt'] 
    full_article_dict['Image url'] = image.find('img')['src']
  lead = soup.find('div', {'class': 'article-body '}).find('h2') # Lead
  if lead:
    full_article_dict['Lead'] = "".join(lead.text.splitlines())
  author = soup.find('div', {'class': 'article-body__author-name'}) # Author
  if author:
    full_article_dict['Author'] = "".join(author.text.splitlines())
  else:
    author = soup.find('div', {'class': 'article-body__byline'}).find('a') # Author
    if author:
      full_article_dict['Author'] = "".join(author.text.splitlines())
  time = soup.find('span', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  time = soup.find('div', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  body = []
  for p in soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'): # Body
    p_text = p.text
    if p_text:
      body.append(p_text)
      body_text = ' '.join(body[:-1]).replace("  ", " ")
      body_text = "".join(body_text.splitlines())
      full_article_dict['Body'] = body_text
  full_articles_list.append(full_article_dict)
  nbr += 1

In [ ]:
# Δημιουργία του dataframe
news247_latest_full_articles_df = pd.DataFrame(full_articles_list)
news247_latest_full_articles_df

,Title,Article url,Section,Image alt text,Image url,Lead,Author,Datetime,Body
0,Ευλογιά των Πιθήκων: Πρώτο επιβεβαιωμένο κρούσ...,https://www.news247.gr/kosmos/eylogia-ton-pith...,Κόσμος,Γιατροί σε εργαστήριο της Γαλλίας,https://www.news247.gr/img/4769/9637672/583000...,Ακόμα μία χώρα στην οποία εντοπίστηκε το πρώτο...,news247,2022-05-20 12:12,Ένα πρώτο επιβεβαιωμένο κρούσμα της ευλογιάς τ...
1,Ο Ούλριχ Ράσε στην Επίδαυρο: Γιατί οι παραστάσ...,https://www.news247.gr/politismos/theatro/o-oy...,Θέατρο,"Ο Γερμανός σκηνοθέτης, Ούλριχ Ράσε",https://www.news247.gr/img/752/9637612/600000/...,Η σκηνή που θα στηθεί στις 22 και 23 Ιουλίου π...,Γεωργία Οικονόμου,2022-05-20 11:50,Πολυαναμενόμενη η παράσταση του καταξιωμένου Γ...
2,Δίκη Τοπαλούδη: Κανένα ελαφρυντικό για τους δο...,https://www.news247.gr/koinonia/diki-topaloydi...,Κοινωνία,Οι δύο κατηγορούμενοι για τη δολοφονία και το ...,https://www.news247.gr/img/36/9637565/787000/w...,"Με ομόφωνη απόφαση, το Μικτό Ορκωτό Δικαστήριο...",Αντωνία Ξυνού,2022-05-20 11:42,Την απόρριψη όλων των ελαφρυντικών περιστάσεων...
3,Θυμίζεις κάτι από Πρέβελη: Το άγνωστο φοινικόδ...,https://www.news247.gr/taxidi/thymizeis-kati-a...,Ταξίδι,Το άγνωστο φοινικόδασος της Αττικής,https://www.news247.gr/img/6760/9637609/291000...,"Περιηγηθείτε σε ένα καταπράσινο φοινικόδασος, ...",news247,2022-05-20 11:40,Αν έχετε επισκεφτεί το φοινικόδασος στην μαγευ...
4,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/koinonia/rethymno-epith...,Κοινωνία,Ρέθυμνο: Επίθεση στο γραμματέα του Δικτύου Φοι...,https://www.news247.gr/img/9061/9637620/890000...,Αγνωστοι εξαπέλυσαν επίθεση με αυγό στο γραμμα...,news247,2022-05-20 11:39,Η ένταση που επικρατεί στο χώρο του Πανεπιστημ...
...,...,...,...,...,...,...,...,...,...
195,Survivor Trailer: Χάος στην παραλία - Ποιος απ...,https://www.news247.gr/psixagogia/tileorasi/su...,Τηλεόραση,Στιγμιότυπο από το Συμβούλιο του Νησιού του Su...,https://www.news247.gr/img/6139/9635912/679000...,Τεράστια θα είναι η ένταση στο αποψινό επεισόδ...,news247,2022-05-19 08:52,Γεμάτο εντάσεις αναμένεται να είναι το νέο επε...
196,Τροχαίο ατύχημα με τρία αυτοκίνητα και τρεις τ...,https://www.news247.gr/koinonia/trochaio-atych...,Κοινωνία,Ασθενοφόρο του ΕΚΑΒ,https://www.news247.gr/img/9585/9629015/814000...,Το περιστατικό συνέβη λίγο πριν τα μεσάνυχτα σ...,Θοδωρής Παναγιωτίδης,2022-05-19 08:40,Τροχαίο ατύχημα σημειώθηκε λίγο πριν τα μεσάνυ...
197,Σκύδρα: Στο Ιπποκράτειο 7χρονος - Έπεσε επάνω ...,https://www.news247.gr/koinonia/skydra-sto-ipp...,Κοινωνία,Το Ιπποκράτειο Νοσοκομείο Θεσσαλονίκης,https://www.news247.gr/img/6906/9558439/146000...,"Σοβαρό ατύχημα σημειώθηκε στη Σκύδρα Πέλλας, ό...",news247,2022-05-19 08:34,Στο Ιπποκράτειο Νοσοκομείο της Θεσσαλονίκης νο...
198,"Τζορτζ Μπους: ""Αδικαιολόγητη και βάναυση η εισ...",https://www.news247.gr/kosmos/gkafa-olkis-o-tz...,Κόσμος,Ο Τζορτζ Ουόκερ Μπους,https://www.news247.gr/img/9619/9635890/302000...,Ο Τζορτζ Μπους έγινε αντικείμενο έντονου σχολι...,news247,2022-05-19 08:22,"Κατά τη διάρκειας ομιλίας του στο Ντάλας, ο πρ..."


##Αυτή τη φορά φαίνονται όλα εντάξει.

In [ ]:
news247_latest_full_articles_df[news247_latest_full_articles_df['Author'].notnull()]

,Title,Article url,Section,Image alt text,Image url,Lead,Datetime,Body,Author
1,Καιρός: Βελτιωμένος στις περισσότερες περιοχές...,https://www.news247.gr/kairos/kairos-veltiomen...,Καιρός,Καιρός,https://www.news247.gr/img/7616/9637180/872000...,Η θερμοκρασία θα ανέβει έως τους 26 βαθμούς Κε...,2022-05-19 22:55,"Την Παρασκευή στις Κυκλάδες, την Κρήτη και τα ...",Θοδωρής Κολυδάς
19,Αυξημένο ενδιαφέρον από Ολλανδούς τουρίστες - ...,https://www.news247.gr/tourismos/ayximeno-endi...,Τουρισμός,Κίνηση στο αεροδρόμιο Ελευθέριος Βενιζέλος,https://www.news247.gr/img/6912/9636509/294000...,Δεύτερο δρομολόγιο προς Ολλανδία από την AEGEA...,2022-05-19 20:53,«Ανεβάζει ταχύτητες» η εισερχόμενη τουριστική ...,Βίκη Βαμιεδάκη
30,15 πράγματα που δε γνωρίζαμε για τον σπουδαίο ...,https://www.news247.gr/politismos/mousiki/15-p...,Μουσική,Ο Βαγγέλης Παπαθανασίου,https://www.news247.gr/img/4473/9636912/750000...,Οι πολυδιάστατες ονειρικές ηλεκτρονικές συνθέσ...,2022-05-19 19:48,Ήταν ένας από τους τελευταίους μεγάλους Έλληνε...,Χριστίνα Τσατσαράγκου...
36,Κικίλιας: Αύξηση κατά 40% των τουριστικών ροών...,https://www.news247.gr/tourismos/kikilias-ayxi...,Τουρισμός,Κικίλιας: Αύξηση κατά 40% των τουριστικών ροών...,https://www.news247.gr/img/4943/9636784/877000...,Αποστολή στο Βελιγράδι πραγματοποιεί ο έλληνας...,2022-05-19 19:08,Το έντονο ενδιαφέρον των Σέρβων ταξιδιωτών για...,Βίκη Βαμιεδάκη
50,ΠΟΜΙΔΑ: Αντισυνταγματικές οι επιβαρύνσεις του ...,https://www.news247.gr/oikonomia/pomida-antisy...,Οικονομία,ΠΟΜΙΔΑ: Αντισυνταγματικές οι επιβαρύνσεις του ...,https://www.news247.gr/img/6059/9636431/246000...,Η ΠΟΜΙΔΑ ενημερώνει τους ιδιοκτήτες ακινήτων π...,2022-05-19 17:20,"""Άδικη αλλά και αντισυνταγματική είναι η διπλή...",Γιώργος Αλεξάκης
59,Τι θα δούμε το φετινό καλοκαίρι στον Κήπο του ...,https://www.news247.gr/politismos/ti-tha-doyme...,Πολιτισμός,Τι θα δούμε το φετινό καλοκαίρι στον Κήπο του ...,https://www.news247.gr/img/3722/9636397/396000...,Ξανανταμώνουμε καθισμένοι στο γρασίδι πάνω σε ...,2022-05-19 16:37,Ο Κήπος του Μεγάρου Μουσικής Αθηνών φωτίζεται ...,Γεωργία Οικονόμου
96,"Στο Ίνκουμπους του Βαγγέλη Γιαννίση, η καταιγι...",https://www.news247.gr/politismos/vivlia/sto-i...,Βιβλία,"Στο Ίνκουμπους του Βαγγέλη Γιαννίση, η καταιγι...",https://www.news247.gr/img/5410/9635082/414000...,Ο Ελληνοσουηδός επιθεωρητής Άντερς Οικονομίδης...,2022-05-19 13:09,«Πιστεύετε ότι έκανε κακό σε εκείνες τις κοπέ...,Γιώργος Ρομπόλας
106,Δύο στρατηγικές παγίδες για την Ευρώπη - Μιλιτ...,https://www.news247.gr/politiki/dyo-stratigike...,Πολιτική,Γερμανικό Eurofighter Typhoon jet στη δύναμη τ...,https://www.news247.gr/img/522/9636113/020000/...,"Η ""βίαιη"" απεξάρτηση από τη ρωσική ενέργεια αν...",2022-05-19 12:01,Οι γεωπολιτικοί και οικονομικοί συσχετισμοί πο...,Βαγγέλης Βιτζηλαίος
108,Μαγνητικά Πεδία: Πώς παραλίγο να μην γυριστεί ...,https://www.news247.gr/sunday-edition/magnitik...,Sunday Edition,Μαγνητικά Πεδία: Πώς παραλίγο να μην γυριστεί ...,https://www.news247.gr/img/2256/9635849/265000...,"Η πρωταγωνίστρια Έλενα Τοπαλίδου, ο σκηνοθέτης...",2022-05-19 11:51,"Για κάτι λίγες στιγμές, εκεί λίγο πριν την Πρω...",Θοδωρής Δημητρόπουλος
113,"Αδιανόητο: Αποκάλεσε αστυνομικό ""καραφλό"" και ...",https://www.news247.gr/koinonia/adianoito-apok...,Κοινωνία,Αστυνομικοί,https://www.news247.gr/img/8854/9636546/143000...,Πώς από μάρτυρας περιστατικού αστυνομικής βίας...,2022-05-19 11:24,Στην Ελλάδα μπορείς να βρεθείς από τη μία στιγ...,Νίκος Γιαννόπουλος


In [ ]:
news247_latest_full_articles_df.groupby('Section').size()

Section
Advertorial          4
Celebrities          7
New Articles         3
Sunday Edition       4
Έγκλημα              2
Αθλητικά             4
Βιβλία               1
Γνώμες               3
Επιστήμη             1
Επιχειρήσεις         2
Θέατρο               1
Καιρός               3
Κοινωνία            40
Κόσμος              46
Λάκης Λαζόπουλος     1
Μουσική              7
Οικονομία           10
Πολιτική            26
Πολιτισμός           2
Σινεμά               2
Τέχνες               1
Ταξίδι               1
Τηλεόραση           19
Τουρισμός            4
Υγεία                6
dtype: int64



---



---



##**Στόχος 2**: Να φτιάξω κώδικα για εβδομαδιαία συλλογή άρθρων από το news247

**Πώς θα το κάνω**:
1. Θα μαζεύω τα δεδομένα των σελίδων με τα teaser articles και θα τα σώζω σε ένα dataframe και στη συνέχεια σε csv
2. Θα συλλέγω όλα τα δεδομένα των ολόκληρων άρθρων (Τίτλος, εικόνα [alt και src], section, lead, body και author) και θα τα σώζω σε ένα dataframe και στη συνέχεια σε csv

##**news247 periodic scrapper**

**Μεταβλητές που ΠΡΕΠΕΙ Nα συμπληρώνοται πάντα στην αρχή**

In [ ]:
# ΗΜΕΡΟΜΗΝΙΑ scraping
today = '2022-05-19'

# ΠΟΣΩΝ ΗΜΕΡΩΝ ΑΡΘΡΑ ΘΕΛΩ ΝΑ ΦΕΡΩ
days_to_scrape = 

**Μεταβλητές που υπολογίζονται αυτόματα**

In [ ]:
csv_name = 'news247_latest_' + today +'.csv'
folder_path = 'gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/news247.gr'
csv_path_and_name = folder_path + '/' + csv_name

last_page = days_to_scrape * 5

**Σύνδεση με το drive**

In [ ]:
from google.colab import drive 
drive.mount('gdrive') 

Mounted at gdrive


**Επισκόπηση περιεχομένων του drive**

In [ ]:
ls 'gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/news247.gr'

**Scraping code**

In [ ]:
# Εισαγωγή βιβλιοθηκών
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

main_url = "https://www.news247.gr/latest/?pages=" # το βασικό url των σελίδων με τα teaser articles

teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

# Scraping των σελίδων Latest
for i in range(1, last_page): 
  page_url = main_url + str(i) 
  page = requests.get(page_url) 
  teaser_soup = BeautifulSoup(page.text, 'html.parser') 
  print(f"Now Scraping page: {i}") 
  teaser_soups_list.append(teaser_soup) 
  time.sleep(2)

# Συλλογή των δεδομένων των teaser articles
teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
for teaser_soup in teaser_soups_list:
  teaser_articles = teaser_soup.find_all('article', {'class': 'teaser__article article article_left '})
  for teaser_article in teaser_articles:
    teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
    h2_a = teaser_article.find('h2').find('a')
    if h2_a:
      teaser_article_dict['Title'] = h2_a['title']
      teaser_article_dict['Url'] = h2_a['href']
    lead = teaser_article.find('p')
    if lead:
      teaser_article_dict['Lead'] = lead.text
    section = teaser_article.find('a', {'class': 'article__section article__tag_section'})
    if section:
      teaser_article_dict['Section'] = section['title']
    datetime = teaser_article.find('time')
    if datetime:
      teaser_article_dict['Datetime'] = datetime['datetime']
    teaser_articles_list.append(teaser_article_dict)

# Δημιουργία του dataframe των teaser articles
news247_latest_teaser_articles_df = pd.DataFrame(teaser_articles_list)
news247_latest_teaser_articles_df

# Scraping όλων των σελίδων των ολόκληρων άρθρων
full_soups_list = [] 
nbr = 0 

for article_url in news247_latest_teaser_articles_df['Url']: 
  nbr += 1 
  full_response = requests.get(article_url) 
  full_soup = BeautifulSoup(full_response.text, 'html.parser')
  full_soups_list.append(full_soup)
  time.sleep(2)
  print(f'Scraping article {nbr}')

# Συλλογή των δεδομένων για τα ολόκληρα άρθρα
full_articles_list = []
nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του news247_latest_teaser_articles_df, να φέρνω και το αντίστοιχο url του άρθρου
for soup in full_soups_list:
  print(f"working on article {news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item()}")
  full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
  title = soup.find('div', {'class': 'article-body '}).find('h1') # Title
  if title:
    full_article_dict['Title'] = "".join(title.text.splitlines()).replace('\t', '')
  full_article_dict['Article url'] = news247_latest_teaser_articles_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου
  section = soup.find('div', {'class': 'article-head'}).find('a') # Section
  if section:
    full_article_dict['Section'] = section['title']
  image = soup.find('figure', {'class': 'article-body__picture'}) # Image
  if image:
    full_article_dict['Image alt text'] = image.find('img')['alt'] 
    full_article_dict['Image url'] = image.find('img')['src']
  lead = soup.find('div', {'class': 'article-body '}).find('h2') # Lead
  if lead:
    full_article_dict['Lead'] = "".join(lead.text.splitlines())
  author = soup.find('div', {'class': 'article-body__author-name'}) # Author
  if author:
    full_article_dict['Author'] = "".join(author.text.splitlines())
  else:
    author = soup.find('div', {'class': 'article-body__byline'}).find('a') # Author
    if author:
      full_article_dict['Author'] = "".join(author.text.splitlines())
  time = soup.find('span', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  time = soup.find('div', {'class': 'article-body__date'}) # Date
  if time:
    full_article_dict['Datetime'] = time.find('time')['datetime']
  body = []
  for p in soup.find('div', {'class': 'article-body__storyContent '}).find_all('p'): # Body
    p_text = p.text
    if p_text:
      body.append(p_text)
      body_text = ' '.join(body[:-1]).replace("  ", " ")
      body_text = "".join(body_text.splitlines())
      full_article_dict['Body'] = body_text
  full_articles_list.append(full_article_dict)
  nbr += 1

# Δημιουργία του dataframe με τα δεδομένα των ολόκληρων άρθρων
news247_latest_full_articles_df = pd.DataFrame(full_articles_list)

**Επισκόπηση του dataframe**

In [ ]:
news247_latest_full_articles_df

**Αποθήκευση του dataframe σαν csv**

In [ ]:
news247_latest_full_articles_df.to_csv(csv_path_and_name, index=False)

# Έλεγχος της αποθήκευσης του csv στο Drive
ls 'gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/news247.gr'

**Προσθήκη των νέων άρθρων στο csv με όλα τα άρθρα**

In [ ]:
# Εισαγωγή του csv με όλα τα άρθρα και μετατροπή του σε dataframe
allnews247_df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/news247.gr/news247_latest_all.csv')
allnews247_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           200 non-null    object
 1   Article url     200 non-null    object
 2   Section         200 non-null    object
 3   Image alt text  200 non-null    object
 4   Image url       200 non-null    object
 5   Lead            200 non-null    object
 6   Author          200 non-null    object
 7   Datetime        200 non-null    object
 8   Body            200 non-null    object
dtypes: object(9)
memory usage: 14.2+ KB


In [ ]:
# Προσθήκη του df των άρθρων που κατεβάσαμε τώρα στο df όλων των άρθρων
allnews247_df = allnews247_df.append(news247_latest_full_articles_df, ignore_index=True)
allnews247_df.info()

In [ ]:
# Aποθήκευση του ενημερωμένου df με όλα τα άρθρα στο Drive σαν csv
news247_latest_full_articles_df.to_csv('gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/news247.gr/news247_latest_all.csv', index=False)